# 52 文本预处理


In [7]:
import collections
import re
from d2l import torch as d2l

加载数据集：

In [8]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


### tokenize

将文本转换为词元序列

In [9]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


### 词汇表

将词映射到从0开始的数字索引

In [10]:
import collections

class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """
        初始化词表
        :param tokens: 输入的词元列表，可以是一维列表或二维列表。默认为空列表。
        :param min_freq: 最小词频阈值, 低于该阈值的词元将不会被加入词表。默认为0。
        :param reserved_tokens: 保留的特殊词元列表，如 '<pad>', '<bos>', '<eos>' 等。默认为空列表。
        """
        # 如果 tokens 为 None，则初始化为空列表
        if tokens is None:
            tokens = []
        # 如果 reserved_tokens 为 None，则初始化为空列表
        if reserved_tokens is None:
            reserved_tokens = []
        # 调用 count_corpus 函数统计词元的频率
        counter = count_corpus(tokens)
        # 按词频从高到低对词元进行排序
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为 0，将 '<unk>' (未知词元)和保留词元添加到 idx_to_token 列表中
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 构建词元到索引的映射字典
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        # 遍历排序后的词元频率列表
        for token, freq in self._token_freqs:
            # 如果词频低于最小词频阈值，则停止添加词元
            if freq < min_freq:
                break
            # 如果词元不在 token_to_idx 字典中，则将其添加到词表中
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        """
        返回词表的大小
        """
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """
        根据词元获取对应的索引
        :param tokens: 输入的词元，可以是单个词元或词元列表
        :return: 对应的索引或索引列表
        """
        # 如果 tokens 不是列表或元组，则直接获取该词元的索引
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        # 如果 tokens 是列表或元组，则递归调用 __getitem__ 方法获取每个词元的索引
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        """
        根据索引获取对应的词元
        :param indices: 输入的索引，可以是单个索引或索引列表
        :return: 对应的词元或词元列表
        """
        # 如果 indices 不是列表或元组，则直接获取该索引对应的词元
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        # 如果 indices 是列表或元组，则遍历每个索引，获取对应的词元
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        """
        返回未知词元的索引
        """
        return 0

    @property
    def token_freqs(self):
        """
        返回词元频率列表
        """
        return self._token_freqs

def count_corpus(tokens):  #@save
    """
    统计词元的频率
    :param tokens: 输入的词元列表，可以是一维列表或二维列表
    :return: 词元频率的计数器对象
    """
    # 如果 tokens 为空或其第一个元素是列表，则将其展平成一维列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    # 使用 collections.Counter 统计词元的频率
    return collections.Counter(tokens)

通过时光机数据集构建词表：

In [11]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以将每一条文本行转换成一个数字索引列表：

In [12]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]
